MICE_SSP245

In [1]:
pwd

'/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final'

In [2]:
!echo $CONDA_DEFAULT_ENV
# to know the current environment

ssdp_divya


In [3]:
import xarray as xr 
print(xr.__version__) 
import pandas as pd 
print(pd.__version__)

2023.4.2
2.0.1


conda install xarray
pip install --upgrade xarray

import xarray as xr
print(xr.__version__)
import pandas as pd
print(pd.__version__)

pip install scikit-learn

pip install hickle

conda install pandarallel

conda install dask

conda install matplotlib

conda install seaborn


conda install tqdm


conda install -c conda-forge netcdf4


conda install -c conda-forge h5netcdf

pip install --upgrade pandas

In [ ]:
# First, We have calculated the benefits, considering the different classes of storage at t and t+1, inflow calsses, months, irrigation and domestic water demand classes 
# we have considered the constant industrial data (http://nca.gov.in/forms_pdf/annual-report/annal-report-english-2020-21.pdf)
# we have considred the 246.7 MCM yearly as allocated for municipal use

In [2]:
import xarray as xr
import random
import numpy as np
import pandas as pd
from random import randrange
#import matplotlib.pyplot as plt
import os
import ast
import time 
import seaborn as sns
#import cartopy.crs as ccrs
import statistics
from matplotlib import gridspec
import glob

#from scipy.stats import iqr
from dateutil import parser
#from matplotlib import pyplot
%matplotlib inline
import itertools
#from joblib import Parallel, delayed
import time
from tqdm import tqdm
#from matplotlib.lines import Line2D
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=40)
pwd_path = os.getcwd()
# weights to different components of objective functions
w1 = 1
w2 = 1
w3 = 1
w4 = 1
w5 = 1



INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Estimate streamflow at Dam location using the hydrological model (VIC in our case) and also estimate the irrigation water demands corresponding all ensembel of MICE and MME
# Calculate the benefits for all the combinations of inflow, storage and irrigation water demand classes

In [ ]:
path1_SSDP_data = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/'
streamflow_SSP585_calculation = '/media/udit/Expansion1/PHD_work/hydropower_work/SSP585_calculation/'
streamflow_SSP245_calculation_MICE = '/media/udit/Expansion1/PHD_work/hydropower_work/Hydropower_MICE_calculation_divya/hydropower_papers/Final_hydropower_data_figures/Narmada_calculation_data/rout_result_narmada/internal_rout_nar_without_biascorrection'
path_irrigation_demand_ssp245_historical_MICE = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MICE_crop_water_requirements/MICE_avg_monthly_cropwater_requirement_cubicmeters_historical_all_realisations.nc'
path_irrigation_demand_ssp245_RCP245_MICE ='/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MICE_crop_water_requirements/MICE_avg_monthly_cropwater_requirement_cubicmeters_rcp245_all_realisations.nc'
streamflow_MME_ssp245 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_montlhy_term_all_initial_condition_narmada_MME_ssp245.nc')
streamflow_MICE_ssp245 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_montlhy_term_all_initial_condition_narmada_MICE_ssp245.nc')
streamflow_MME_ssp585 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_term_all_initial_condition_narmada_MME_ssp585.nc')
streamflow_MICE_ssp585 = xr.open_dataset(path1_SSDP_data +'streamflow_full_term_all_initial_condition_narmada_MICE_ssp585.nc')
domestic_demand= xr.open_dataset(path1_SSDP_data +'drinking_water_requirements_1901_2101_data.nc')
nc_fpaths = [path_irrigation_demand_ssp245_historical_MICE,path_irrigation_demand_ssp245_RCP245_MICE]
irrigation_water_demand_MICE = xr.open_mfdataset(nc_fpaths, coords='minimal',  engine='netcdf4')
benefit_calculated = xr.open_dataset(path1_SSDP_data +'benefit_SSDP_different_combinations_nonnegetive_release_irrigation_case1.nc')

## benefit calculation for 2100

In [5]:
bins_irrigation=[0, 150, 295, 336,380,410, 440, 480,630,710,750, 780, 840,965, 1050,  1090,1120, 1180,1270,1450, 1700, 2000,2150,2440,2800, 3350 ]
#bins = (np.array(bins)/1000000) # in MCM
irrigation_demand_classes1 = []
irrigation_demand_representative_values1 = []
for class1 in range(len(bins_irrigation) - 1):
    class2 = [bins_irrigation[class1],bins_irrigation[class1+1]]
    representative_values = (bins_irrigation[class1]+bins_irrigation[class1+1])/2
    #print(representative_values)
    irrigation_demand_representative_values1.append(representative_values)
    irrigation_demand_classes1.append(class2)
print(len(irrigation_demand_representative_values1))

bins_inflow=[0,3,6,20, 50, 100,150, 200,300,400,600,800,1000,1200,1500, 2000,2700, 3500,4500, 5000,6000, 7000,8000,9400,10500,12000,15000,20000, 30000,40000,48300]
inflow_classes1 = []
inflow_representative_values1 = []
for class1 in range(len(bins_inflow) - 1):
    class2 = [bins_inflow[class1],bins_inflow[class1+1]]
    representative_values = (bins_inflow[class1]+bins_inflow[class1+1])/2
    #print(representative_values)
    inflow_representative_values1.append(representative_values)
    inflow_classes1.append(class2)
print(len(inflow_representative_values1))
bins_storage=[0.002,0.3,0.45,0.6,0.75,0.9,1.05,1.14
                                        ,1.2,1.3,1.37, 1.42, 1.48, 1.535, 1.57,2,2.5, 3.01823,3.5, 3.991,4.4,4.8,5, 5.4, 5.65,5.76]
print(len(bins_storage))
bins_storage= np.array(bins_storage)* 1000
storage_classes = []
storage_representative_values = []
for class1 in range(len(bins_storage) - 1):
    class2 = [bins_storage[class1],bins_storage[class1+1]]
    representative_values = (bins_storage[class1]+bins_storage[class1+1])/2
    #print(representative_values)
    storage_representative_values.append(representative_values)
    storage_classes.append(class2)
      
domestic_demand = xr.open_dataset('drinking_water_requirements_1901_2101_data.nc')

25
30
26


In [6]:
def class_representative_values(inflow1, inflow_classes1,inflow_representative_values1):
    for i in range(len(inflow_classes1)):
        #print(inflow1)
        if inflow_classes1[i][0] <= inflow1 and inflow_classes1[i][1] > inflow1:
            inflow_representative_values2 = inflow_representative_values1[i]
    return(inflow_representative_values2)
        
def functional_value_calc_2100(S_t, S_t_1, inflow_t,drinking_water_demand,irrigation_water_demand, month):
    #inflow_t_month = class_representative_values(inflow_t, inflow_classes1, inflow_representative_values1)
    #drinking_water_demand_month = class_representative_values(irrigation_water_demand,irrigation_demand_classes1, irrigation_demand_representative_values1)
    #irrigation_water_demand_month = class_representative_values(irrigation_water_demand,irrigation_demand_classes1, irrigation_demand_representative_values1)
    benefit = benefit_calculated['benefit_calculated'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    Release_t = benefit_calculated['Release_t'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    #print(Release_t)
    drinking_water_release = benefit_calculated['drinking_water_release'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    irrigation_water_release = benefit_calculated['irrigation_water_release'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    Industrial_water_release = benefit_calculated['Industrial_water_release'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    power_production = benefit_calculated['power_production'].sel(S_t= S_t,S_t_1=S_t_1,inflow_t = inflow_t,drinking_water_demand1 = drinking_water_demand,irrigation_water_demand1 = irrigation_water_demand,month = month)
    return [benefit, Release_t,drinking_water_release,irrigation_water_release,Industrial_water_release, power_production]

# Calculation for MICE_ssp245

In [ ]:
streamflow_MICE_ssp245 = xr.open_dataset('streamflow_full_montlhy_term_all_initial_condition_narmada_MICE_ssp245.nc')
inflow_full_monthly = streamflow_MICE_ssp245
irrigation_water_demand = irrigation_water_demand_MICE
benefit_2100 = pd.DataFrame({'initial_condition':[],'storage_t':[], 'storage_t_1':[],'month':[],'release_t':[],'drinking_water_release':[],'irrigation_water_release':[],'Industrial_water_release':[],'power_production':[],'benefit':[]})
initial_condition =streamflow_MICE_ssp245['initial_condition'].values
for j, initial_condition1 in tqdm(itertools.islice(enumerate(initial_condition),0,None,1)):
    for k,storage_t in tqdm(itertools.islice(enumerate(storage_representative_values),0,None,1)):
        for l, storage_t_1 in tqdm(itertools.islice(enumerate(storage_representative_values),0,None,1)):
            for month in tqdm(itertools.islice(range(12),0,None,1)):
                domestic_demand_data = (domestic_demand['Drinking_water_demand'].sel(year = 2100).values)/1000000
                #domestic_demand_data = class_representative_values(domestic_demand_data, domestic_demand_classes1,domestic_demand_representative_values1)
                #print(initial_condition1)
                #print(domestic_demand_data)
                if initial_condition1 == 'EC-EARTH3':
                    initial_condition2 = 'EC-Earth3'
                    irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition2, year = 2100,month =month+1).values)/1000000
                    irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
                else:
                    irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition1, year = 2100,month =month+1).values)/1000000
                    irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
                #print(initial_condition1)
                if initial_condition1 == 'EC-EARTH3':
                    initial_condition2 = 'EC-EARTH3'
                    inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition2, year = 2100,month =month+1).values
                    inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
                else:
                    inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition1, year = 2100,month =month+1).values
                    inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
                #print(inflow_t1)
                #print(irrigation_water_demand_data)
                [benefit1, Release_t,drinking_water_release,irrigation_water_release,Industrial_water_release, power_production] = functional_value_calc_2100(storage_t, storage_t_1, inflow_t1,domestic_demand_data,irrigation_water_demand_data, month)
                #print(Release_t)
                benefit_2100 = benefit_2100.append({'initial_condition':initial_condition1,'storage_t':storage_t, 'storage_t_1':storage_t_1,'month':month,'release_t':Release_t.values,'drinking_water_release':drinking_water_release.values,'irrigation_water_release':irrigation_water_release.values,'Industrial_water_release':Industrial_water_release.values,'power_production':power_production.values,'benefit':benefit1.values},ignore_index=True) 
                #print(benefit_2100)
benefit_2100 = benefit_2100.set_index(['initial_condition','storage_t','storage_t_1','month'])
df2 = benefit_2100.to_xarray().to_netcdf(pwd_path+'/benefit_2100_final_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')
#benefit_2100.to_xarray().to_netcdf('/benefit_2100_final_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')

df3 = xr.open_dataset(pwd_path+'/benefit_2100_final_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')
optimised_value_2100 = pd.DataFrame({'initial_condition':[],'storage_t':[], 'month':[],'optimised_function_value':[],'optimised_storage_t':[],'optimised_drinking_release':[],'optimised_irrigation_release':[],'optimised_industrial_release':[],'power_production':[]})
for j, initial_condition1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
    for k,storage_t in tqdm(itertools.islice(enumerate(df3['storage_t'].values),0,None,1)): 
        s_t_initial = storage_t
        for month in tqdm(itertools.islice(range(12),0,None,1)):
            index = np.argmin(df3['benefit'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values)
            max_values = df3['benefit'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values[index]
            opti_drinking = df3['drinking_water_release'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values[index]
            opti_irrigation = df3['irrigation_water_release'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values[index]
            opti_industrial = df3['Industrial_water_release'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values[index]
            opti_power_production = df3['power_production'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month).values[index]
            
        #print(max_values)
            s_t_1 = df3['benefit'].sel(initial_condition = initial_condition1, storage_t = s_t_initial,month =month)[index]['storage_t_1'].values
            s_t_initial = s_t_1
            optimised_value_2100 = optimised_value_2100.append({'initial_condition':initial_condition1,'storage_t':storage_t,'month':month+1,'optimised_function_value':max_values,'optimised_storage_t': s_t_1,'optimised_drinking_release':opti_drinking,'optimised_irrigation_release':opti_irrigation,'optimised_industrial_release':opti_industrial,'power_production':opti_power_production},ignore_index=True)
optimised_value_2100_1 = optimised_value_2100.set_index(['initial_condition','storage_t','month'])
optimised_value_2100_1.to_xarray().to_netcdf('optimised_value_2100_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')


abc = xr.open_dataset('optimised_value_2100_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')
plt.close('all')
plt.rcParams.update({'font.size': 22})
sns.set_style("whitegrid")
#projection=ccrs.Orthographic(-90, 85)
fig = plt.figure(figsize=(20,13))
gs = gridspec.GridSpec(2,3)
ax0 = fig.add_subplot(gs[0])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][0], storage_t = abc['storage_t'][i]).plot()
    
ax0 = fig.add_subplot(gs[1])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][1], storage_t = abc['storage_t'][i]).plot()
ax0 = fig.add_subplot(gs[2])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][2], storage_t = abc['storage_t'][i]).plot()
ax0 = fig.add_subplot(gs[3])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][3], storage_t = abc['storage_t'][i]).plot()
ax0 = fig.add_subplot(gs[4])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][4], storage_t = abc['storage_t'][i]).plot()
    
ax0 = fig.add_subplot(gs[5])
for i in range(10):
    abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][5], storage_t = abc['storage_t'][i]).plot()


In [ ]:
# Calculation for 2099

In [9]:
pwd_path

'/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final'

In [ ]:
abc = xr.open_dataset(pwd_path+'/optimised_value_2100_with20storage_nonnegetive_release_MME_SSP245_irrigation_case1.nc')
initial_condition = abc['initial_condition'].values
# benefit_combinations1 = pd.DataFrame({'S_t':[],'S_t_1':[],'initial_condition':[],'month':[]})

# for i,S_t in tqdm(itertools.islice(enumerate(storage_representative_values1),0,None,1)):
#     for j,S_t_1 in tqdm(itertools.islice(enumerate(storage_representative_values1),0,None,1)):
#         for k,initial_condition1 in tqdm(itertools.islice(enumerate(initial_condition),0,None,1)):
#             for month in tqdm(itertools.islice(range(12),0,None,1)):
#                 benefit_combinations1 = benefit_combinations1.append({'S_t':S_t,'S_t_1':S_t_1,'initial_condition':initial_condition1,'month':month},ignore_index=True) 
 
# benefit_combinations_1 = pd.DataFrame({'S_t':[],'S_t_1':[]})
# for i,S_t in tqdm(itertools.islice(enumerate(storage_representative_values),0,None,1)):
#     for j,S_t_1 in tqdm(itertools.islice(enumerate(storage_representative_values),0,None,1)):
#         benefit_combinations_1 = benefit_combinations_1.append({'S_t':S_t,'S_t_1':S_t_1},ignore_index=True)
# abc_1 = benefit_combinations_1.set_index(['S_t','S_t_1'])
# abc_1.to_xarray().to_netcdf('combination_s_t_and_s_T_1_allcases.nc')        
    
# benefit_combinations_1 = pd.DataFrame({'initial_condition':[],'month':[]})
# for k,initial_condition1 in tqdm(itertools.islice(enumerate(initial_condition),0,None,1)):
#     for month in tqdm(itertools.islice(range(12),0,None,1)):
#         benefit_combinations_1 = benefit_combinations_1.append({'initial_condition':initial_condition1,'month':month},ignore_index=True)
# abc_1 = benefit_combinations_1.set_index(['initial_condition','month'])
# abc_1.to_xarray().to_netcdf('combination_initial_condition_month_MME.nc')
                                                                
import netCDF4
from netCDF4 import Dataset

dataset = xr.open_mfdataset(['combination_s_t_and_s_T_1_allcases.nc', 'combination_initial_condition_month_MME.nc'], concat_dim='None',combine='nested')
dataset.to_netcdf('benefit_combinations_for_SSDP_with20storage_MME.nc')                  

In [5]:
abc = xr.open_dataset(pwd_path+'/optimised_value_2100_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc')
abc11 = xr.open_dataset(pwd_path+'/benefit_combinations_for_SSDP_with20storage_MICE.nc')
abc11 = abc11.to_dataframe().reset_index()
len(np.unique(abc11['S_t']))
final_T_P = xr.open_dataset(pwd_path+'/SSDP_transition_probability_nonBC_with6models_MICE_SSP245.nc')

In [8]:
len(np.unique(abc11['initial_condition']))

49

# Separate calculation for benefit calculation

In [4]:
modellists = glob.glob(pwd_path+ '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_*.nc')
print((modellists))


['/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/optimised_values_MICE_6models_SSP245_irrigation_case1/benefit_value_1995_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc', '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/optimised_values_MICE_6models_SSP245_irrigation_case1/benefit_value_2074_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc', '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/optimised_values_MICE_6models_SSP245_irrigation_case1/benefit_value_2077_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc', '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/optimised_values_MICE_6models_SSP245_irrigation_case1/benefit_value_2017_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc', '/home/udit/Desktop/Data_MIR_students/P

In [8]:
modellists = glob.glob(pwd_path+ '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_*.nc')
print(len(modellists))
modellists1 = glob.glob(pwd_path+ '/optimised_values_MICE_6models_SSP245_irrigation_case1/optimised_value_*.nc')
print(len(modellists1))
inflow_full_monthly = streamflow_MICE_ssp245
years1 = list(range(1980,2100))
for i in reversed(years1):
    #print(i)
    file_exists = False
    for number in range(len(modellists)):
        if int(modellists[number].split('/')[-1].split('_')[2]) == i:
            print("i =", i)
            print('benefit_File exists')
            file_exists = True
            break
    if not file_exists:
        #print('calculation')
        if i < 2015:
            irrigation_water_demand = xr.open_dataset(path_irrigation_demand_ssp245_historical_MICE)
            #print(irrigation_water_demand['year'])
        else:
            irrigation_water_demand = xr.open_dataset(path_irrigation_demand_ssp245_RCP245_MICE)
        def irrigation_demand_1(row):
            initial_condition2 = row['initial_condition']
            #print(initial_condition2)
            if initial_condition2 == 'EC-EARTH3':
                initial_condition2 = 'EC-Earth3'
            year1 = i
            irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition2, year = year1,month =row['month']+1).values)/1000000
            irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
            return irrigation_water_demand_data

        def inflow_1(row):
            year1 = i
            initial_condition1 = row['initial_condition']

            if initial_condition1 == 'EC-Earth3':
                initial_condition2 = 'EC-EARTH3'
                inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition2, year = year1,month =row['month']+1).values
                inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
                return inflow_t1
            else:
                inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition1, year = year1,month =row['month']+1).values
                inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
                return inflow_t1

        def benefit_1(row):
            domestic_demand_data1 = (domestic_demand['Drinking_water_demand'].sel(year = i).values)/1000000
            #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            #domestic_demand_data1 = domestic_demand_data
            benefit = benefit_calculated['benefit_calculated'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return benefit
        def drinking_demand_1(row):
            #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            domestic_demand_data1 = domestic_demand_data
            benefit = benefit_calculated['drinking_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return benefit
        def irrigation_demand_2(row):
            #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            domestic_demand_data1 = domestic_demand_data
            benefit = benefit_calculated['irrigation_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return benefit
        def industrial_demand_1(row):
            #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            domestic_demand_data1 = domestic_demand_data
            benefit = benefit_calculated['Industrial_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return benefit
        def power_production_1(row):
            #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            domestic_demand_data1 = domestic_demand_data
            benefit = benefit_calculated['power_production'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return benefit

        def release_1(row):
           # benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
            domestic_demand_data1 = domestic_demand_data
            Release_t = benefit_calculated['Release_t'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
            return Release_t

        def second_term_1(row):
            previous_functional_values = 'optimised_value_' + str(i+1) +'.nc'
            abc_1 = xr.open_dataset(previous_functional_values)
            second_term = np.sum(abc_1['optimised_function_value'].sel(initial_condition = row['initial_condition'],storage_t =  row['S_t_1']).values)
            return second_term
        abc11 = xr.open_dataset(pwd_path + '/benefit_combinations_for_SSDP_with20storage_MICE.nc')
        abc11 = abc11.to_dataframe().reset_index()
        domestic_demand_data = (domestic_demand['Drinking_water_demand'].sel(year = i).values)/1000000
        #domestic_demand_data = class_representative_values(domestic_demand_data, domestic_demand_classes1,domestic_demand_representative_values1)

        abc11['irrigation_demand'] = abc11.parallel_apply(irrigation_demand_1, axis=1)
        #print(abc11)
        abc11['inflow_1'] = abc11.parallel_apply(inflow_1, axis=1)
        abc11['benefit'] = abc11.parallel_apply(benefit_1, axis=1)
        abc11['drinking_water_release'] = abc11.parallel_apply(drinking_demand_1, axis=1)
        abc11['irrigation_water_release'] = abc11.parallel_apply(irrigation_demand_2, axis=1)
        abc11['Industrial_water_release'] = abc11.parallel_apply(industrial_demand_1, axis=1)
        abc11['power_production'] = abc11.parallel_apply(power_production_1, axis=1)
        abc11['release_1'] = abc11.parallel_apply(release_1, axis=1)
        #print(abc11)
        #abc11['second_term_1'] = abc11.parallel_apply(second_term_1, axis=1)
        benefit_1 = abc11.set_index(['S_t', 'S_t_1','initial_condition','month'])
        benefit_values_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        benefit_1.to_xarray().to_netcdf(benefit_values_name)
        del(benefit_1)
        del(abc11)


120
96
i = 2099
benefit_File exists
i = 2098
benefit_File exists
i = 2097
benefit_File exists
i = 2096
benefit_File exists
i = 2095
benefit_File exists
i = 2094
benefit_File exists
i = 2093
benefit_File exists
i = 2092
benefit_File exists
i = 2091
benefit_File exists
i = 2090
benefit_File exists
i = 2089
benefit_File exists
i = 2088
benefit_File exists
i = 2087
benefit_File exists
i = 2086
benefit_File exists
i = 2085
benefit_File exists
i = 2084
benefit_File exists
i = 2083
benefit_File exists
i = 2082
benefit_File exists
i = 2081
benefit_File exists
i = 2080
benefit_File exists
i = 2079
benefit_File exists
i = 2078
benefit_File exists
i = 2077
benefit_File exists
i = 2076
benefit_File exists
i = 2075
benefit_File exists
i = 2074
benefit_File exists
i = 2073
benefit_File exists
i = 2072
benefit_File exists
i = 2071
benefit_File exists
i = 2070
benefit_File exists
i = 2069
benefit_File exists
i = 2068
benefit_File exists
i = 2067
benefit_File exists
i = 2066
benefit_File exists
i = 206

# Separate Calculation for optimised Values


In [9]:
        def optimised_term1(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =1).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =2).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =3).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =4).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial,month =5).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month = 6).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =7).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =8).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =9).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =10).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month = 11).values[max_index]
            s_t_initial = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =11)[max_index]['S_t_1'].values

            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            first_term2_1 = np.sum(first_term2)
            scaled_first_term2_1 = first_term2_1/(10 ** 10)
            return scaled_first_term2_1
        
        def optimised_term1_storage(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            s_t_initial = [s_t_initial_1m, s_t_initial_2m,s_t_initial_3m, s_t_initial_4m,s_t_initial_5m, s_t_initial_6m,s_t_initial_7m, s_t_initial_8m, s_t_initial_9m, s_t_initial_10m,s_t_initial_11m, s_t_initial_12m]
            return s_t_initial

        def optimised_term1_drinking_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = df3['drinking_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            return first_term2

        def optimised_term1_irrigation_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = df3['irrigation_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            return first_term2

        def optimised_term1_Industrial_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = df3['Industrial_water_release'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            return first_term2

        def optimised_term1_power_production(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = df3['power_production'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            return first_term2


        def optimised_term1_benefit(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            #df3 = xr.open_dataset(benefit_values_name)            
            # month1
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial, month=0).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_1m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial,month =0).values[max_index]
            s_t_initial_1m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =0)[max_index]['S_t_1'].values
            # month2
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_1m, month=1).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_2m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_1m,month =1).values[max_index]
            s_t_initial_2m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_1m,month =1)[max_index]['S_t_1'].values
   
            # month3
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_2m, month=2).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_3m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_2m,month =2).values[max_index]
            s_t_initial_3m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_2m,month =2)[max_index]['S_t_1'].values
   
            # month4
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_3m, month=3).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_4m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_3m,month =3).values[max_index]
            s_t_initial_4m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_3m,month = 3)[max_index]['S_t_1'].values

            # month5
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_4m, month=4).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_5m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_4m,month =4).values[max_index]
            s_t_initial_5m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_4m,month =4)[max_index]['S_t_1'].values

            # month6
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_5m, month=5).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_6m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t =s_t_initial_5m,month =5).values[max_index]
            s_t_initial_6m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_5m,month = 5)[max_index]['S_t_1'].values

            # month7
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_6m, month=6).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_7m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_6m,month = 6).values[max_index]
            s_t_initial_7m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_6m,month =6)[max_index]['S_t_1'].values

            # month8
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_7m, month=7).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_8m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_7m,month =7).values[max_index]
            s_t_initial_8m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_7m,month =7)[max_index]['S_t_1'].values

            # month9
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_8m, month=8).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_9m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_8m,month =8).values[max_index]
            s_t_initial_9m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_8m,month =8)[max_index]['S_t_1'].values

            # month10
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_9m, month=9).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_10m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_9m,month =9).values[max_index]
            s_t_initial_10m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_9m,month = 9)[max_index]['S_t_1'].values

            # month11
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_10m, month=10).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_11m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_10m,month =10).values[max_index]
            s_t_initial_11m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_10m,month = 10)[max_index]['S_t_1'].values

            # month12
            data_index = -(df3['benefit'].sel(initial_condition=row['initial_condition'], S_t=s_t_initial_11m, month=11).values)
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0]
            first_term2_12m = -df3['benefit'].sel(initial_condition = row['initial_condition'], S_t = s_t_initial_11m,month = 11).values[max_index]
            s_t_initial_12m = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial_11m,month =11)[max_index]['S_t_1'].values
            first_term2 = [first_term2_1m, first_term2_2m,first_term2_3m, first_term2_4m,first_term2_5m, first_term2_6m,first_term2_7m, first_term2_8m, first_term2_9m, first_term2_10m,first_term2_11m, first_term2_12m]
            return first_term2



In [10]:
        def optimised_term2_index(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            #df3 = xr.open_dataset(benefit_values_name)  
            if i <= 2098:
                second_term_1 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][0],storage_t =  s_t_1).values)
                second_term_2 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][1],storage_t =  s_t_1).values)
                second_term_3 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][2],storage_t =  s_t_1).values)
                second_term_4 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][3],storage_t =  s_t_1).values)
                second_term_5 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][4],storage_t =  s_t_1).values)
                second_term_6 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][5],storage_t =  s_t_1).values)
                second_term_7 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][6],storage_t =  s_t_1).values)
                second_term_8 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][7],storage_t =  s_t_1).values)
                second_term_9 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][8],storage_t =  s_t_1).values)
                second_term_10 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][9],storage_t =  s_t_1).values)
                second_term_11 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][10],storage_t =  s_t_1).values)
                second_term_12 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][11],storage_t =  s_t_1).values)
                second_term_13 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][12],storage_t =  s_t_1).values)
                second_term_14 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][13],storage_t =  s_t_1).values)
                second_term_15 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][14],storage_t =  s_t_1).values)
                second_term_16 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][15],storage_t =  s_t_1).values)
                second_term_17 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][16],storage_t =  s_t_1).values)
                second_term_18 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][17],storage_t =  s_t_1).values)
                second_term_19 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][18],storage_t =  s_t_1).values)
                second_term_20 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][19],storage_t =  s_t_1).values)
                second_term_21 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][20],storage_t =  s_t_1).values)
                second_term_22 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][21],storage_t =  s_t_1).values)
                second_term_23 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][22],storage_t =  s_t_1).values)
                second_term_24 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][23],storage_t =  s_t_1).values)
                second_term_25 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][24],storage_t =  s_t_1).values)
                second_term_26 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][25],storage_t =  s_t_1).values)
                second_term_27 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][26],storage_t =  s_t_1).values)
                second_term_28 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][27],storage_t =  s_t_1).values)
                second_term_29 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][28],storage_t =  s_t_1).values)
                second_term_30 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][29],storage_t =  s_t_1).values)
                second_term_31 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][30],storage_t =  s_t_1).values)
                second_term_32 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][31],storage_t =  s_t_1).values)
                second_term_33 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][32],storage_t =  s_t_1).values)
                second_term_34 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][33],storage_t =  s_t_1).values)
                second_term_35 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][34],storage_t =  s_t_1).values)
                second_term_36 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][35],storage_t =  s_t_1).values)
                second_term_37 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][36],storage_t =  s_t_1).values)
                second_term_38 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][37],storage_t =  s_t_1).values)
                second_term_39 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][38],storage_t =  s_t_1).values)
                second_term_40 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][39],storage_t =  s_t_1).values)
                second_term_41 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][40],storage_t =  s_t_1).values)
                second_term_42 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][41],storage_t =  s_t_1).values)
                second_term_43 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][42],storage_t =  s_t_1).values)
                second_term_44 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][43],storage_t =  s_t_1).values)
                second_term_45 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][44],storage_t =  s_t_1).values)
                second_term_46 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][45],storage_t =  s_t_1).values)
                second_term_47 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][46],storage_t =  s_t_1).values)
                second_term_48 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][47],storage_t =  s_t_1).values)
                second_term_49 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][48],storage_t =  s_t_1).values)
            else: 
                second_term_1 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][0],storage_t =  s_t_1).values)
                second_term_2 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][1],storage_t =  s_t_1).values)
                second_term_3 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][2],storage_t =  s_t_1).values)
                second_term_4 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][3],storage_t =  s_t_1).values)
                second_term_5 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][4],storage_t =  s_t_1).values)
                second_term_6 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][5],storage_t =  s_t_1).values)
                second_term_7 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][6],storage_t =  s_t_1).values)
                second_term_8 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][7],storage_t =  s_t_1).values)
                second_term_9 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][8],storage_t =  s_t_1).values)
                second_term_10 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][9],storage_t =  s_t_1).values)
                second_term_11 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][10],storage_t =  s_t_1).values)
                second_term_12 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][11],storage_t =  s_t_1).values)
                second_term_13 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][12],storage_t =  s_t_1).values)
                second_term_14 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][13],storage_t =  s_t_1).values)
                second_term_15 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][14],storage_t =  s_t_1).values)
                second_term_16 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][15],storage_t =  s_t_1).values)
                second_term_17 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][16],storage_t =  s_t_1).values)
                second_term_18 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][17],storage_t =  s_t_1).values)
                second_term_19 =-np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][18],storage_t =  s_t_1).values)
                second_term_20 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][19],storage_t =  s_t_1).values)
                second_term_21 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][20],storage_t =  s_t_1).values)
                second_term_22 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][21],storage_t =  s_t_1).values)
                second_term_23 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][22],storage_t =  s_t_1).values)
                second_term_24 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][23],storage_t =  s_t_1).values)
                second_term_25 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][24],storage_t =  s_t_1).values)
                second_term_26 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][25],storage_t =  s_t_1).values)
                second_term_27 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][26],storage_t =  s_t_1).values)
                second_term_28 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][27],storage_t =  s_t_1).values)
                second_term_29 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][28],storage_t =  s_t_1).values)
                second_term_30 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][29],storage_t =  s_t_1).values)
                second_term_31 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][30],storage_t =  s_t_1).values)
                second_term_32 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][31],storage_t =  s_t_1).values)
                second_term_33 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][32],storage_t =  s_t_1).values)
                second_term_34 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][33],storage_t =  s_t_1).values)
                second_term_35 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][34],storage_t =  s_t_1).values)
                second_term_36 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][35],storage_t =  s_t_1).values)
                second_term_37 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][36],storage_t =  s_t_1).values)
                second_term_38 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][37],storage_t =  s_t_1).values)
                second_term_39 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][38],storage_t =  s_t_1).values)
                second_term_40 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][39],storage_t =  s_t_1).values)
                second_term_41 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][40],storage_t =  s_t_1).values)
                second_term_42 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][41],storage_t =  s_t_1).values)
                second_term_43 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][42],storage_t =  s_t_1).values)
                second_term_44 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][43],storage_t =  s_t_1).values)
                second_term_45 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][44],storage_t =  s_t_1).values)
                second_term_46 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][45],storage_t =  s_t_1).values)
                second_term_47 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][46],storage_t =  s_t_1).values)
                second_term_48 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][47],storage_t =  s_t_1).values)
                second_term_49 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][48],storage_t =  s_t_1).values)
                
            second_term2 = [second_term_1, second_term_2, second_term_3, second_term_4,second_term_5, second_term_6, second_term_7, second_term_8, second_term_9, second_term_10, 
                          second_term_11, second_term_12, second_term_13, second_term_14,second_term_15, second_term_16, second_term_17, second_term_18, second_term_19, second_term_20,
                          second_term_21, second_term_22, second_term_23, second_term_24,second_term_25, second_term_26, second_term_27, second_term_28, second_term_29, second_term_30,
                          second_term_31, second_term_32, second_term_33, second_term_34,second_term_35, second_term_36, second_term_37, second_term_38, second_term_39, second_term_40,
                          second_term_41, second_term_42, second_term_43, second_term_44,second_term_45, second_term_46, second_term_47, second_term_48, second_term_49]
            data_index = second_term2
            max_value = max(data_index)
            max_index = np.argwhere(data_index == max_value)
            max_index  = max_index [-1][0] 
            return max_index

        def optimised_term2(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                second_term2_1 = np.unique(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t = s_t_1).values)         
            else:
                second_term2_1 = -np.sum(abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t = s_t_1).values)         

            scaled_second_term2_1 = second_term2_1/(10 ** 10)
            return scaled_second_term2_1

        def optimised_term2_storage(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                second_term_storage = abc['optimised_storage_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            else: 
                second_term_storage = abc['optimised_storage_t'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       

            return second_term_storage

        def optimised_term2_drinking_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                drinking_1 = abc['optimised_drinking_release_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            else: 
                drinking_1 = abc['optimised_drinking_release'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            return drinking_1

        def optimised_term2_irrigation_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                irrigation_1 = abc['optimised_irrigation_release_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            else: 
                irrigation_1 = abc['optimised_irrigation_release'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            return irrigation_1

        def optimised_term2_Industrial_water(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                Industrial_1 = abc['optimised_industrial_release_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            else: 
                Industrial_1 = abc['optimised_industrial_release'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            return Industrial_1

        def optimised_term2_power_production(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                power_production_1 = abc['power_production_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            else: 
                power_production_1 = abc['power_production'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values       
            return power_production_1

        def optimised_term2_benefit(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            if i <= 2098:
                benefit_1 = abc['optimised_function_series'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values
            else:
                benefit_1 = -abc['optimised_function_value'].sel(initial_condition = streamflow_MICE_ssp585['initial_condition'][max_index],storage_t =  s_t_1).values
            return benefit_1

        def optimised_transition_probability(row):
            s_t_initial = row['S_t']
            initial_condition1 = row['initial_condition']
            s_t_1 = row['optimised_term1_storage'][-1]
            max_index = row['scaled_first_index']
            initial_condition_1 = abc['initial_condition'][max_index]
            if initial_condition1 == 'EC-Earth3':
                initial_condition2 = 'EC-EARTH3'
            else:
                initial_condition2 = initial_condition1
            
            if initial_condition_1 == 'EC-Earth3':
                initial_condition_2 = 'EC-EARTH3'
            else:
                initial_condition_2 = initial_condition_1
            transition_probability = final_T_P['TP_final'].sel(year =i,initial_i = initial_condition2, initial_j = initial_condition_2).values
            return transition_probability
        
        def optimised_values_final(row):
            s_t_initial = row['S_t']
            scaled_first_term2_1 = row['scaled_first_term2_1']
            transition_probability = row['transition_probability']
            scaled_second_term2_1 = row['scaled_second_term2_1']
            optimised_values = scaled_first_term2_1 + (transition_probability * scaled_second_term2_1)
            return optimised_values
        
        def optimised_series_final(row):
            optimised_series = row['optimised_term1_benefit']+ row['optimised_term2_benefit'].tolist()
            return optimised_series 
        
        def optimised_storage_series_final(row):
            optimised_storage_series = row['optimised_term1_storage']+ row['optimised_term2_storage'].tolist()
            return optimised_storage_series  


        def optimised_drinking_water_series_final(row):
            optimised_drinking_water_series = row['optimised_term1_drinking_water']+ row['optimised_term2_drinking_water'].tolist()
            return optimised_drinking_water_series  


        def optimised_irrigation_water_series_final(row):
            optimised_irrigation_water_series = row['optimised_term1_irrigation_water']+ row['optimised_term2_irrigation_water'].tolist()
            return optimised_irrigation_water_series  


        def optimised_Industrial_water_series_final(row):
            optimised_Industrial_water_series = row['optimised_term1_Industrial_water']+ row['optimised_term2_Industrial_water'].tolist()
            return optimised_Industrial_water_series  

        def optimised_power_production_series_final(row):
            optimised_power_production_series = row['optimised_term1_power_production']+ row['optimised_term2_power_production'].tolist()
            return optimised_power_production_series  
        


In [11]:
modellists1 = glob.glob(pwd_path+ '/optimised_values_MICE_6models_SSP245_irrigation_case1/optimised_value_*.nc')
print(len(modellists1))
inflow_full_monthly = streamflow_MICE_ssp245
years1 = list(range(1981,2100))
for i in reversed(years1):
    #print(i)
    file_exists = False
    for number in range(len(modellists1)):
        if int(modellists1[number].split('/')[-1].split('_')[2]) == i:
            print("i =", i)
            print('optimised_File exists')
            file_exists = True
            break
    if not file_exists:
        print('calculation')
        benefit_values_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        df3 = xr.open_dataset(benefit_values_name)
        if i+1 < 2100:
            previous_functional_values = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
            abc = xr.open_dataset(previous_functional_values)

        else:
            previous_functional_values = pwd_path + '/optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
            abc = xr.open_dataset(previous_functional_values)
        # calculation for the first term for all combinations of initial conditions and the storages
        abc12 = xr.open_dataset(pwd_path+'/combination_initial_condition_S_t_MICE.nc')
        abc12 = abc12.to_dataframe().reset_index()
        abc12['scaled_first_term2_1'] = abc12.parallel_apply(optimised_term1, axis=1)
        abc12['optimised_term1_storage'] = abc12.parallel_apply(optimised_term1_storage, axis=1)
        abc12['optimised_term1_drinking_water'] = abc12.parallel_apply(optimised_term1_drinking_water, axis=1)
        abc12['optimised_term1_irrigation_water'] = abc12.parallel_apply(optimised_term1_irrigation_water, axis=1)
        abc12['optimised_term1_Industrial_water'] = abc12.parallel_apply(optimised_term1_Industrial_water, axis=1)
        abc12['optimised_term1_power_production'] = abc12.parallel_apply(optimised_term1_power_production, axis=1)
        abc12['optimised_term1_benefit'] = abc12.parallel_apply(optimised_term1_benefit, axis=1)

     # calculation for the first term for all combinations of initial conditions and the storages

        abc12['scaled_first_index'] = abc12.parallel_apply(optimised_term2_index, axis=1)
        abc12['scaled_second_term2_1'] = abc12.parallel_apply(optimised_term2, axis=1)
        abc12['optimised_term2_storage'] = abc12.parallel_apply(optimised_term2_storage, axis=1)
        abc12['optimised_term2_drinking_water'] = abc12.parallel_apply(optimised_term2_drinking_water, axis=1)
        abc12['optimised_term2_irrigation_water'] = abc12.parallel_apply(optimised_term2_irrigation_water, axis=1)
        abc12['optimised_term2_Industrial_water'] = abc12.parallel_apply(optimised_term2_Industrial_water, axis=1)
        abc12['optimised_term2_power_production'] = abc12.parallel_apply(optimised_term2_power_production, axis=1)
        abc12['optimised_term2_benefit'] = abc12.parallel_apply(optimised_term2_benefit, axis=1)

        abc12['transition_probability'] = abc12.parallel_apply(optimised_transition_probability, axis=1)
        abc12['optimised_values'] = abc12.parallel_apply(optimised_values_final, axis=1)
        abc12['optimised_series'] = abc12.parallel_apply(optimised_series_final, axis=1)
        abc12['optimised_storage_series'] = abc12.parallel_apply(optimised_storage_series_final, axis=1)
        abc12['optimised_drinking_water_series'] = abc12.parallel_apply(optimised_drinking_water_series_final, axis=1)
        abc12['optimised_irrigation_water_series'] = abc12.parallel_apply(optimised_irrigation_water_series_final, axis=1)
        abc12['optimised_Industrial_water_series'] = abc12.parallel_apply(optimised_Industrial_water_series_final, axis=1)
        abc12['optimised_power_production_series'] = abc12.parallel_apply(optimised_power_production_series_final, axis=1)

        index_number = len(abc12['optimised_irrigation_water_series'][0])
        repeated_condition = np.repeat(abc12['initial_condition'], index_number)
        repeated_st = np.repeat(abc12['S_t'], index_number)
        repeated_index = np.tile(np.arange(len(abc12['optimised_irrigation_water_series'][0])), (len(abc12['S_t'])))
        optimised_values = np.repeat(abc12['optimised_values'], index_number)
        optimised_series=np.concatenate(abc12['optimised_series'])
        optimised_storage_series=np.concatenate(abc12['optimised_storage_series'])
        opti_drinking_series=np.concatenate(abc12['optimised_drinking_water_series'])
        opti_irrigation_series=np.concatenate(abc12['optimised_irrigation_water_series'])
        opti_industrial_series=np.concatenate(abc12['optimised_Industrial_water_series'])
        opti_power_production_series=np.concatenate(abc12['optimised_power_production_series'])
        #
        print('done')
        optimised_value_1_2 = pd.DataFrame({'initial_condition':repeated_condition,'storage_t':repeated_st,'index_number':repeated_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series,'optimised_storage_series':optimised_storage_series,'optimised_drinking_release_series':opti_drinking_series,'optimised_irrigation_release_series':opti_irrigation_series,'optimised_industrial_release_series':opti_industrial_series,'power_production_series':opti_power_production_series})
        
#         for j, initial_condition1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
#             for k,storage_t in tqdm(itertools.islice(enumerate(df3['S_t'].values),0,None,1)): 
#                 filtered_data = abc12[(abc12['initial_condition'] == initial_condition1) & (abc12['S_t'] == storage_t)]     
#                 optimised_values = filtered_data['optimised_values'].values
#                 optimised_series = filtered_data['optimised_series'].values
#                 optimised_storage_series = filtered_data['optimised_storage_series'].values
#                 opti_drinking_series = filtered_data['optimised_drinking_water_series'].values
#                 opti_irrigation_series = filtered_data['optimised_irrigation_water_series'].values
#                 opti_industrial_series = filtered_data['optimised_Industrial_water_series'].values
#                 opti_power_production_series= filtered_data['optimised_power_production_series'].values
#                 for number_index in range(len(opti_drinking_series[0])):
#                     optimised_value_1_2 = pd.DataFrame({'initial_condition':initial_condition1,'storage_t':storage_t,'index_number':number_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series[0][number_index],'optimised_storage_series':optimised_storage_series[0][number_index],'optimised_drinking_release_series':opti_drinking_series[0][number_index],'optimised_irrigation_release_series':opti_irrigation_series[0][number_index],'optimised_industrial_release_series':opti_industrial_series[0][number_index],'power_production_series':opti_power_production_series[0][number_index]}, index = [0])
#                    # print(storage_t)

#                     if j == 0 and k == 0 and number_index == 0:
#                         concatenated_df = optimised_value_1_2
#                         optimised_value_1_1 = concatenated_df
#                         #print(optimised_value_1_1)
#                         print(1)
#                     else:
#                         concatenated_df = pd.concat([optimised_value_1_1, optimised_value_1_2])
#                         #print(optimised_value_1_1)
#                         optimised_value_1_1 = concatenated_df
#                         #print(optimised_value_1_1)
#                         #print(2)

#                 #optimised_value_1 = optimised_value_1.append({'initial_condition':initial_condition1,'storage_t':storage_t,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series,'optimised_storage_series':optimised_storage_series,'optimised_drinking_release_series':opti_drinking_series,'optimised_irrigation_release_series':opti_irrigation_series,'optimised_industrial_release_series':opti_industrial_series,'power_production_series':opti_power_production_series},ignore_index=True)

        pkl_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        optimised_value_1_2.set_index(['initial_condition','storage_t','index_number']).to_xarray().to_netcdf(pkl_name)
        del(optimised_value_1_2)
        del(abc12)

96
i = 2099
optimised_File exists
i = 2098
optimised_File exists
i = 2097
optimised_File exists
i = 2096
optimised_File exists
i = 2095
optimised_File exists
i = 2094
optimised_File exists
i = 2093
optimised_File exists
i = 2092
optimised_File exists
i = 2091
optimised_File exists
i = 2090
optimised_File exists
i = 2089
optimised_File exists
i = 2088
optimised_File exists
i = 2087
optimised_File exists
i = 2086
optimised_File exists
i = 2085
optimised_File exists
i = 2084
optimised_File exists
i = 2083
optimised_File exists
i = 2082
optimised_File exists
i = 2081
optimised_File exists
i = 2080
optimised_File exists
i = 2079
optimised_File exists
i = 2078
optimised_File exists
i = 2077
optimised_File exists
i = 2076
optimised_File exists
i = 2075
optimised_File exists
i = 2074
optimised_File exists
i = 2073
optimised_File exists
i = 2072
optimised_File exists
i = 2071
optimised_File exists
i = 2070
optimised_File exists
i = 2069
optimised_File exists
i = 2068
optimised_File exists
i = 2

In [ ]:
modellists1 = glob.glob(pwd_path+ '/optimised_values_MICE_6models_SSP245_irrigation_case1/optimised_value_*.nc')
print(len(modellists1))
inflow_full_monthly = streamflow_MICE_ssp245
years1 = list(range(1980,2100))
for i in reversed(years1):
    #print(i)
    file_exists = False
    for number in range(len(modellists1)):
        if int(modellists1[number].split('/')[-1].split('_')[2]) == i:
            print("i =", i)
            print('optimised_File exists')
            file_exists = True
            break
    if not file_exists:
        #print('calculation')
        benefit_values_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        df3 = xr.open_dataset(benefit_values_name)
        #print(i)
        if i+1 < 2100:
            previous_functional_values = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
            abc = xr.open_dataset(previous_functional_values)

        else:
            previous_functional_values = pwd_path + '/optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
            abc = xr.open_dataset(previous_functional_values)

        for j, initial_condition1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
            for k,storage_t in tqdm(itertools.islice(enumerate(df3['S_t'].values),0,None,1)): 
                #print(storage_t)
                s_t_initial = storage_t
                s_t_2 = []
                first_term2 =[]
                first_term_drinking = []
                first_term_irrigation = []
                first_term_industrial = []
                first_term_power_production = []
                for month in tqdm(itertools.islice(range(12),0,None,1)):
                    #index = np.argmax(-df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values)
                    data_index = -(df3['benefit'].sel(initial_condition=initial_condition1, S_t=s_t_initial, month=month).values)
                    max_value = max(data_index)
                    min_value = min(data_index)

                    max_index = next(i for i, value in enumerate(data_index) if value == max_value)
                    min_index = next(i for i, value in enumerate(data_index) if value == min_value)

                    min_values = -df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]

                    opti_drinking = df3['drinking_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                    opti_irrigation = df3['irrigation_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                    opti_industrial = df3['Industrial_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                    opti_power_production = df3['power_production'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]

                    s_t_1 = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month)[max_index]['S_t_1'].values
                    s_t_initial = s_t_1
                    s_t_2.append(s_t_1)
                    first_term2.append(min_values)
                    first_term_drinking.append(opti_drinking)
                    first_term_irrigation.append(opti_irrigation)
                    first_term_industrial.append(opti_industrial)
                    first_term_power_production.append(opti_power_production)
                first_term2_1 = np.sum(first_term2) 
                scaled_first_term2_1 = first_term2_1/(10 ** 10)

               # print(i)

                if i <= 2098:
                    second_term2 =[]
                    #print(abc)
                    for l, initial_condition_1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
                        second_term = np.unique(abc['optimised_function_value'].sel(initial_condition = initial_condition_1,storage_t =  s_t_1).values)
                        second_term2.append(second_term)
                   # print(second_term2)
                    # index = np.argmin(second_term2)
                    data_index = second_term2
                    max_value = max(data_index)
                    min_value = min(data_index)

                    max_index = next(i for i, value in enumerate(data_index) if value == max_value)
                    min_index = next(i for i, value in enumerate(data_index) if value == min_value)

                    #print(index)
                    min_values_1 = abc['optimised_function_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                    drinking_1 = abc['optimised_drinking_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                    irrigation_1 = abc['optimised_irrigation_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                    industrial_1 = abc['optimised_industrial_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                    power_production_1 = abc['power_production_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values

                    #plt.plot(min_values_1[0])
                    second_term2_1 = []
                    second_term2_1 = np.unique(abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values)         
                    second_term_storage = abc['optimised_storage_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values       
                    if initial_condition1 == 'EC-Earth3':
                        initial_condition2 = 'EC-EARTH3'
                    else:
                        initial_condition2 = initial_condition1

                    if initial_condition_1 == 'EC-Earth3':
                        initial_condition_2 = 'EC-EARTH3'
                    else:
                        initial_condition_2 = initial_condition_1

                    transition_probability = final_T_P['TP_final'].sel(year =i,initial_i = initial_condition2, initial_j = initial_condition_2).values
                    scaled_second_term2_1 = second_term2_1/(10 ** 10)
                    optimised_values = scaled_first_term2_1 + (transition_probability * scaled_second_term2_1)
                   # print(optimised_values)
                    optimised_series = first_term2 +min_values_1.tolist()
                    #plt.plot(optimised_series)
                    optimised_storage_series = s_t_2 + second_term_storage.tolist()
                    opti_drinking_series = first_term_drinking+drinking_1.tolist()
                    opti_irrigation_series = first_term_irrigation+irrigation_1.tolist()
                    opti_industrial_series = first_term_industrial+industrial_1.tolist()
                    opti_power_production_series = first_term_power_production+power_production_1.tolist()
                    for number_index in range(len(opti_drinking_series)):
                        optimised_value_1_2 = pd.DataFrame({'initial_condition':initial_condition1,'storage_t':storage_t,'index_number':number_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series[number_index],'optimised_storage_series':optimised_storage_series[number_index],'optimised_drinking_release_series':opti_drinking_series[number_index],'optimised_irrigation_release_series':opti_irrigation_series[number_index],'optimised_industrial_release_series':opti_industrial_series[number_index],'power_production_series':opti_power_production_series[number_index]}, index = [0])
                   # print(storage_t)

                        if j == 0 and k == 0 and number_index == 0:
                            concatenated_df = optimised_value_1_2
                            optimised_value_1_1 = concatenated_df
                            #print(concatenated_df)
                        else:
                            concatenated_df = pd.concat([optimised_value_1_1, optimised_value_1_2])
                        #print(optimised_value_1_1)
                        optimised_value_1_1 = concatenated_df
                        del(optimised_value_1_2)
                        del(concatenated_df)
                else:
                    #print(abc)
                    second_term2 =[]
                    for l, initial_condition_1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
                        second_term = -np.sum(abc['optimised_function_value'].sel(initial_condition = initial_condition_1,storage_t =  s_t_1).values)
                        second_term2.append(second_term)
                    index = np.argmax(second_term2)
                    min_values_1 = -abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                    drinking_1 = abc['optimised_drinking_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                    irrigation_1 = abc['optimised_irrigation_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                    industrial_1 = abc['optimised_industrial_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                    power_production_1 = abc['power_production'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values

                    plt.plot(min_values_1)
                    second_term2_1 = []
                    second_term2_1 = -np.sum(abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values)         
                   # print(second_term2_1)
                    second_term_storage = abc['optimised_storage_t'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values       
                    if initial_condition1 == 'EC-Earth3':
                        initial_condition2 = 'EC-EARTH3'
                    else:
                        initial_condition2 = initial_condition1

                    if initial_condition_1 == 'EC-Earth3':
                        initial_condition_2 = 'EC-EARTH3'
                    else:
                        initial_condition_2 = initial_condition_1

                    transition_probability = final_T_P['TP_final'].sel(year =i,initial_i = initial_condition2, initial_j = initial_condition_2).values                
                    scaled_second_term2_1 = second_term2_1/(10 ** 10)
                    optimised_values = scaled_first_term2_1 + (transition_probability * scaled_second_term2_1)
                   # print(optimised_values)
                    optimised_series = first_term2 +list(min_values_1)
                    #plt.plot(optimised_series)
                    optimised_storage_series = s_t_2 + list(second_term_storage) 
                    opti_drinking_series = first_term_drinking+list(drinking_1)
                    #print(opti_drinking_series)
                    opti_irrigation_series = first_term_irrigation+list(irrigation_1)
                    opti_industrial_series = first_term_industrial+list(industrial_1)
                    opti_power_production_series = first_term_power_production+list(power_production_1)
                    for number_index in range(len(opti_drinking_series)):
                        optimised_value_1_2 = pd.DataFrame({'initial_condition':initial_condition1,'storage_t':storage_t,'index_number':number_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series[number_index],'optimised_storage_series':optimised_storage_series[number_index],'optimised_drinking_release_series':opti_drinking_series[number_index],'optimised_irrigation_release_series':opti_irrigation_series[number_index],'optimised_industrial_release_series':opti_industrial_series[number_index],'power_production_series':opti_power_production_series[number_index]}, index = [0])
                   # print(storage_t)

                        if i == 2099 and j == 0 and k == 0 and number_index == 0:
                            concatenated_df = optimised_value_1_2
                            optimised_value_1_1 = concatenated_df
                            #print(optimised_value_1_1)
                        else:
                            concatenated_df = pd.concat([optimised_value_1_1, optimised_value_1_2])
                        #print(optimised_value_1_1)
                        optimised_value_1_1 = concatenated_df

                #optimised_value_1 = optimised_value_1.append({'initial_condition':initial_condition1,'storage_t':storage_t,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series,'optimised_storage_series':optimised_storage_series,'optimised_drinking_release_series':opti_drinking_series,'optimised_irrigation_release_series':opti_irrigation_series,'optimised_industrial_release_series':opti_industrial_series,'power_production_series':opti_power_production_series},ignore_index=True)

        pkl_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        optimised_value_1_1.set_index(['initial_condition','storage_t','index_number']).to_xarray().to_netcdf(pkl_name)
        del(optimised_value_1_1)
        

In [ ]:
# full calculation including the benefit calculation for each year and the optimised values calculations

In [ ]:
inflow_full_monthly = streamflow_MICE_ssp245
years1 = list(range(1980,2100))
for i in reversed(years1):
    print(i)
    if i < 2015:
        irrigation_water_demand = xr.open_dataset(path_irrigation_demand_ssp245_historical_MICE)
        #print(irrigation_water_demand['year'])
    else:
        irrigation_water_demand = xr.open_dataset(path_irrigation_demand_ssp245_RCP245_MICE)
    def irrigation_demand_1(row):
        initial_condition2 = row['initial_condition']
        #print(initial_condition2)
        if initial_condition2 == 'EC-EARTH3':
            initial_condition2 = 'EC-Earth3'
        year1 = i
        irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition2, year = year1,month =row['month']+1).values)/1000000
        irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
        return irrigation_water_demand_data

    def inflow_1(row):
        year1 = i
        initial_condition1 = row['initial_condition']

        if initial_condition1 == 'EC-Earth3':
            initial_condition2 = 'EC-EARTH3'
            inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition2, year = year1,month =row['month']+1).values
            inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
            return inflow_t1
        else:
            inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition1, year = year1,month =row['month']+1).values
            inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
            return inflow_t1
    
    def benefit_1(row):
        domestic_demand_data1 = (domestic_demand['Drinking_water_demand'].sel(year = i).values)/1000000
        #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        #domestic_demand_data1 = domestic_demand_data
        benefit = benefit_calculated['benefit_calculated'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return benefit
    def drinking_demand_1(row):
        #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        domestic_demand_data1 = domestic_demand_data
        benefit = benefit_calculated['drinking_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return benefit
    def irrigation_demand_2(row):
        #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        domestic_demand_data1 = domestic_demand_data
        benefit = benefit_calculated['irrigation_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return benefit
    def industrial_demand_1(row):
        #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        domestic_demand_data1 = domestic_demand_data
        benefit = benefit_calculated['Industrial_water_release'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return benefit
    def power_production_1(row):
        #benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        domestic_demand_data1 = domestic_demand_data
        benefit = benefit_calculated['power_production'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return benefit

    def release_1(row):
       # benefit_calculated = xr.open_dataset(pwd_path + '/benefit_SSDP_different_combinations_final2_with20storage_nonnegetive_release_MICE_SSP245_irrigation_constrain.nc')
        domestic_demand_data1 = domestic_demand_data
        Release_t = benefit_calculated['Release_t'].sel(S_t= row['S_t'],S_t_1=row['S_t_1'],inflow_t = row['inflow_1'],drinking_water_demand1 = domestic_demand_data1,irrigation_water_demand1 = row['irrigation_demand'],month = row['month']).values
        return Release_t

    def second_term_1(row):
        previous_functional_values = 'optimised_value_' + str(i+1) +'.nc'
        abc_1 = xr.open_dataset(previous_functional_values)
        second_term = np.sum(abc_1['optimised_function_value'].sel(initial_condition = row['initial_condition'],storage_t =  row['S_t_1']).values)
        return second_term
    abc11 = xr.open_dataset(pwd_path + '/benefit_combinations_for_SSDP_with20storage_MICE.nc')
    abc11 = abc11.to_dataframe().reset_index()
    domestic_demand_data = (domestic_demand['Drinking_water_demand'].sel(year = i).values)/1000000
    #domestic_demand_data = class_representative_values(domestic_demand_data, domestic_demand_classes1,domestic_demand_representative_values1)

    abc11['irrigation_demand'] = abc11.parallel_apply(irrigation_demand_1, axis=1)
    #print(abc11)
    abc11['inflow_1'] = abc11.parallel_apply(inflow_1, axis=1)
    abc11['benefit'] = abc11.parallel_apply(benefit_1, axis=1)
    abc11['drinking_water_release'] = abc11.parallel_apply(drinking_demand_1, axis=1)
    abc11['irrigation_water_release'] = abc11.parallel_apply(irrigation_demand_2, axis=1)
    abc11['Industrial_water_release'] = abc11.parallel_apply(industrial_demand_1, axis=1)
    abc11['power_production'] = abc11.parallel_apply(power_production_1, axis=1)
    abc11['release_1'] = abc11.parallel_apply(release_1, axis=1)
    print(abc11)
    #abc11['second_term_1'] = abc11.parallel_apply(second_term_1, axis=1)
    benefit_1 = abc11.set_index(['S_t', 'S_t_1','initial_condition','month'])
    benefit_values_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'benefit_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
    benefit_1.to_xarray().to_netcdf(benefit_values_name)
    del(benefit_1)
    del(abc11)
    df3 = xr.open_dataset(benefit_values_name)
    print(i)
    
    if i+1 < 2100:
        previous_functional_values = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        abc = xr.open_dataset(previous_functional_values)

    else:
        previous_functional_values = pwd_path + '/optimised_value_' + str(i+1) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
        abc = xr.open_dataset(previous_functional_values)

    for j, initial_condition1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
        for k,storage_t in tqdm(itertools.islice(enumerate(df3['S_t'].values),0,None,1)): 
            #print(storage_t)
            s_t_initial = storage_t
            s_t_2 = []
            first_term2 =[]
            first_term_drinking = []
            first_term_irrigation = []
            first_term_industrial = []
            first_term_power_production = []
            for month in tqdm(itertools.islice(range(12),0,None,1)):
                #index = np.argmax(-df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values)
                data_index = -(df3['benefit'].sel(initial_condition=initial_condition1, S_t=s_t_initial, month=month).values)
                max_value = max(data_index)
                min_value = min(data_index)

                max_index = next(i for i, value in enumerate(data_index) if value == max_value)
                min_index = next(i for i, value in enumerate(data_index) if value == min_value)

                min_values = -df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                
                opti_drinking = df3['drinking_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                opti_irrigation = df3['irrigation_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                opti_industrial = df3['Industrial_water_release'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]
                opti_power_production = df3['power_production'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month).values[max_index]

                s_t_1 = df3['benefit'].sel(initial_condition = initial_condition1, S_t = s_t_initial,month =month)[max_index]['S_t_1'].values
                s_t_initial = s_t_1
                s_t_2.append(s_t_1)
                first_term2.append(min_values)
                first_term_drinking.append(opti_drinking)
                first_term_irrigation.append(opti_irrigation)
                first_term_industrial.append(opti_industrial)
                first_term_power_production.append(opti_power_production)
            first_term2_1 = np.sum(first_term2) 
            scaled_first_term2_1 = first_term2_1/(10 ** 10)

           # print(i)
            
            if i <= 2098:
                second_term2 =[]
                #print(abc)
                for l, initial_condition_1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
                    second_term = np.unique(abc['optimised_function_value'].sel(initial_condition = initial_condition_1,storage_t =  s_t_1).values)
                    second_term2.append(second_term)
               # print(second_term2)
                # index = np.argmin(second_term2)
                data_index = second_term2
                max_value = max(data_index)
                min_value = min(data_index)

                max_index = next(i for i, value in enumerate(data_index) if value == max_value)
                min_index = next(i for i, value in enumerate(data_index) if value == min_value)

                #print(index)
                min_values_1 = abc['optimised_function_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                drinking_1 = abc['optimised_drinking_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                irrigation_1 = abc['optimised_irrigation_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                industrial_1 = abc['optimised_industrial_release_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values
                power_production_1 = abc['power_production_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values

                #plt.plot(min_values_1[0])
                second_term2_1 = []
                second_term2_1 = np.unique(abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values)         
                second_term_storage = abc['optimised_storage_series'].sel(initial_condition = abc['initial_condition'][max_index],storage_t =  s_t_1).values       
                if initial_condition1 == 'EC-Earth3':
                    initial_condition2 = 'EC-EARTH3'
                else:
                    initial_condition2 = initial_condition1
                    
                if initial_condition_1 == 'EC-Earth3':
                    initial_condition_2 = 'EC-EARTH3'
                else:
                    initial_condition_2 = initial_condition_1
                    
                transition_probability = final_T_P['TP_final'].sel(year =i,initial_i = initial_condition2, initial_j = initial_condition_2).values
                scaled_second_term2_1 = second_term2_1/(10 ** 10)
                optimised_values = scaled_first_term2_1 + (transition_probability * scaled_second_term2_1)
               # print(optimised_values)
                optimised_series = first_term2 +min_values_1.tolist()
                #plt.plot(optimised_series)
                optimised_storage_series = s_t_2 + second_term_storage.tolist()
                opti_drinking_series = first_term_drinking+drinking_1.tolist()
                opti_irrigation_series = first_term_irrigation+irrigation_1.tolist()
                opti_industrial_series = first_term_industrial+industrial_1.tolist()
                opti_power_production_series = first_term_power_production+power_production_1.tolist()
                for number_index in range(len(opti_drinking_series)):
                    optimised_value_1_2 = pd.DataFrame({'initial_condition':initial_condition1,'storage_t':storage_t,'index_number':number_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series[number_index],'optimised_storage_series':optimised_storage_series[number_index],'optimised_drinking_release_series':opti_drinking_series[number_index],'optimised_irrigation_release_series':opti_irrigation_series[number_index],'optimised_industrial_release_series':opti_industrial_series[number_index],'power_production_series':opti_power_production_series[number_index]}, index = [0])
               # print(storage_t)
               
                    if j == 0 and k == 0 and number_index == 0:
                        concatenated_df = optimised_value_1_2
                        optimised_value_1_1 = concatenated_df
                        #print(concatenated_df)
                    else:
                        concatenated_df = pd.concat([optimised_value_1_1, optimised_value_1_2])
                    #print(optimised_value_1_1)
                    optimised_value_1_1 = concatenated_df
            else:
                #print(abc)
                second_term2 =[]
                for l, initial_condition_1 in tqdm(itertools.islice(enumerate(df3['initial_condition'].values),0,None,1)):
                    second_term = -np.sum(abc['optimised_function_value'].sel(initial_condition = initial_condition_1,storage_t =  s_t_1).values)
                    second_term2.append(second_term)
                index = np.argmax(second_term2)
                min_values_1 = -abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                drinking_1 = abc['optimised_drinking_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                irrigation_1 = abc['optimised_irrigation_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                industrial_1 = abc['optimised_industrial_release'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values
                power_production_1 = abc['power_production'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values

                plt.plot(min_values_1)
                second_term2_1 = []
                second_term2_1 = -np.sum(abc['optimised_function_value'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values)         
               # print(second_term2_1)
                second_term_storage = abc['optimised_storage_t'].sel(initial_condition = abc['initial_condition'][index],storage_t =  s_t_1).values       
                if initial_condition1 == 'EC-Earth3':
                    initial_condition2 = 'EC-EARTH3'
                else:
                    initial_condition2 = initial_condition1
                    
                if initial_condition_1 == 'EC-Earth3':
                    initial_condition_2 = 'EC-EARTH3'
                else:
                    initial_condition_2 = initial_condition_1
                    
                transition_probability = final_T_P['TP_final'].sel(year =i,initial_i = initial_condition2, initial_j = initial_condition_2).values                
                scaled_second_term2_1 = second_term2_1/(10 ** 10)
                optimised_values = scaled_first_term2_1 + (transition_probability * scaled_second_term2_1)
               # print(optimised_values)
                optimised_series = first_term2 +list(min_values_1)
                #plt.plot(optimised_series)
                optimised_storage_series = s_t_2 + list(second_term_storage) 
                opti_drinking_series = first_term_drinking+list(drinking_1)
                #print(opti_drinking_series)
                opti_irrigation_series = first_term_irrigation+list(irrigation_1)
                opti_industrial_series = first_term_industrial+list(industrial_1)
                opti_power_production_series = first_term_power_production+list(power_production_1)
                for number_index in range(len(opti_drinking_series)):
                    optimised_value_1_2 = pd.DataFrame({'initial_condition':initial_condition1,'storage_t':storage_t,'index_number':number_index,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series[number_index],'optimised_storage_series':optimised_storage_series[number_index],'optimised_drinking_release_series':opti_drinking_series[number_index],'optimised_irrigation_release_series':opti_irrigation_series[number_index],'optimised_industrial_release_series':opti_industrial_series[number_index],'power_production_series':opti_power_production_series[number_index]}, index = [0])
               # print(storage_t)
               
                    if i == 2099 and j == 0 and k == 0 and number_index == 0:
                        concatenated_df = optimised_value_1_2
                        optimised_value_1_1 = concatenated_df
                        #print(optimised_value_1_1)
                    else:
                        concatenated_df = pd.concat([optimised_value_1_1, optimised_value_1_2])
                    #print(optimised_value_1_1)
                    optimised_value_1_1 = concatenated_df
            
            #optimised_value_1 = optimised_value_1.append({'initial_condition':initial_condition1,'storage_t':storage_t,'optimised_function_value':optimised_values,'optimised_function_series':optimised_series,'optimised_storage_series':optimised_storage_series,'optimised_drinking_release_series':opti_drinking_series,'optimised_irrigation_release_series':opti_irrigation_series,'optimised_industrial_release_series':opti_industrial_series,'power_production_series':opti_power_production_series},ignore_index=True)
    
    pkl_name = pwd_path + '/optimised_values_MICE_6models_SSP245_irrigation_case1/'+'optimised_value_' + str(i) +'_with20storage_nonnegetive_release_MICE_SSP245_irrigation_case1.nc'
    optimised_value_1_1.set_index(['initial_condition','storage_t','index_number']).to_xarray().to_netcdf(pkl_name)
    del(optimised_value_1_1)
    del(optimised_value_1_2)
    del(concatenated_df)